## Importing Libraries

In [65]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from datetime import datetime
from sklearn.metrics import accuracy_score

## Reading data into csv files

In [66]:
f_tdata= pd.read_csv('Data_Train.csv',parse_dates=["Date_of_Journey","Arrival_Time","Dep_Time"])
dft=pd.read_csv('Test_set.csv',parse_dates=["Date_of_Journey","Arrival_Time","Dep_Time"])

### Describes (incomplete) data


In [67]:
f_tdata.describe()

,Price
count,10683.000000
mean,9087.064121
std,4611.359167
min,1759.000000
25%,5277.000000
50%,8372.000000
75%,12373.000000
max,79512.000000


In [69]:
dft.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,2019-06-06,Delhi,Cochin,DEL → BOM → COK,2019-06-26 17:30:00,2019-06-07 04:25:00,10h 55m,1 stop,No info
1,IndiGo,2019-12-05,Kolkata,Banglore,CCU → MAA → BLR,2019-06-26 06:20:00,2019-06-26 10:20:00,4h,1 stop,No info
2,Jet Airways,2019-05-21,Delhi,Cochin,DEL → BOM → COK,2019-06-26 19:15:00,2019-05-22 19:00:00,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,2019-05-21,Delhi,Cochin,DEL → BOM → COK,2019-06-26 08:00:00,2019-06-26 21:00:00,13h,1 stop,No info
4,Air Asia,2019-06-24,Banglore,Delhi,BLR → DEL,2019-06-26 23:55:00,2019-06-25 02:45:00,2h 50m,non-stop,No info


Head of Data

In [70]:
f_tdata.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,2019-06-26 22:20:00,2019-03-22 01:10:00,2h 50m,non-stop,No info,3897.0
1,Air India,2019-01-05,Kolkata,Banglore,CCU → IXR → BBI → BLR,2019-06-26 05:50:00,2019-06-26 13:15:00,7h 25m,2 stops,No info,7662.0
2,Jet Airways,2019-09-06,Delhi,Cochin,DEL → LKO → BOM → COK,2019-06-26 09:25:00,2019-06-10 04:25:00,19h,2 stops,No info,13882.0
3,IndiGo,2019-12-05,Kolkata,Banglore,CCU → NAG → BLR,2019-06-26 18:05:00,2019-06-26 23:30:00,5h 25m,1 stop,No info,6218.0
4,IndiGo,2019-01-03,Banglore,New Delhi,BLR → NAG → DEL,2019-06-26 16:50:00,2019-06-26 21:35:00,4h 45m,1 stop,No info,13302.0


Null Values Across DataFrames

In [96]:
dft['Additional_Info']=dft['Additional_Info'].fillna("No Info")

Null Values for "Total Stops" columns<br>
Note that we find the null value for route as well

In [72]:
f_tdata[f_tdata['Total_Stops'].isnull()]


,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
9039,Air India,2019-06-05,Delhi,Cochin,NaN,2019-06-26 09:45:00,2019-05-07 09:25:00,23h 40m,NaN,No info,7480.0


Finding other similar entries

In [73]:
f_tdata.loc[(f_tdata['Airline']=='Air India')&(f_tdata['Source']=='Delhi')&(f_tdata['Destination']=='Cochin')&(f_tdata['Price']==7480.0)&(f_tdata['Dep_Time']=='09:45')]

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
1616,Air India,2019-05-24,Delhi,Cochin,DEL → MAA → COK,2019-06-26 09:45:00,2019-05-25 09:25:00,23h 40m,1 stop,No info,7480.0
2554,Air India,2019-05-15,Delhi,Cochin,DEL → MAA → COK,2019-06-26 09:45:00,2019-05-16 09:25:00,23h 40m,1 stop,No info,7480.0
4351,Air India,2019-06-27,Delhi,Cochin,DEL → MAA → COK,2019-06-26 09:45:00,2019-06-28 09:25:00,23h 40m,1 stop,No info,7480.0
7896,Air India,2019-12-06,Delhi,Cochin,DEL → MAA → COK,2019-06-26 09:45:00,2019-06-13 09:25:00,23h 40m,1 stop,No info,7480.0
9039,Air India,2019-06-05,Delhi,Cochin,NaN,2019-06-26 09:45:00,2019-05-07 09:25:00,23h 40m,NaN,No info,7480.0


Filling Missing Values

In [74]:
f_tdata.iloc[9039,4]="DEL → MAA → COK"

In [75]:
f_tdata.iloc[9039,8]= '1 stop'

### Checking datatypes of f_tdata

In [76]:
f_tdata.dtypes

Airline                    object
Date_of_Journey    datetime64[ns]
Source                     object
Destination                object
Route                      object
Dep_Time           datetime64[ns]
Arrival_Time       datetime64[ns]
Duration                   object
Total_Stops                object
Additional_Info            object
Price                     float64
dtype: object

### Converting f_tdata to date-time and making date of journey ordinal.

In [77]:
f_tdata['Arrival_Time'] = f_tdata['Arrival_Time'].dt.time
f_tdata['Dep_Time'] = f_tdata['Dep_Time'].dt.time


In [79]:
f_tdata['Date_of_Journey']=f_tdata.apply(lambda x:str(x['Date_of_Journey'].toordinal()),axis=1)
dft['Date_of_Journey']=dft.apply(lambda x:str(x['Date_of_Journey'].toordinal()),axis=1)

### Seeing which columns contain categorical data and converting them using OneHotEncoder Scheme.

In [80]:
f_tdata.columns

Index(['Airline', 'Date_of_Journey', 'Source', 'Destination', 'Route',
       'Dep_Time', 'Arrival_Time', 'Duration', 'Total_Stops',
       'Additional_Info', 'Price'],
      dtype='object')

In [81]:
doj_le=LabelEncoder()
doj_labels = doj_le.fit_transform(f_tdata['Date_of_Journey'])
doj_ohe=OneHotEncoder()
doj_feature_arr = doj_ohe.fit_transform(
                              f_tdata[['Date_of_Journey']]).toarray()
doj_feature_labels = list(doj_le.classes_)
doj_features = pd.DataFrame(doj_feature_arr,
                            columns=doj_feature_labels)
f_tdata= pd.concat([f_tdata,doj_features],axis=1)
line_le = LabelEncoder()
line_labels = line_le.fit_transform(f_tdata['Airline'])
line_ohe=OneHotEncoder()
line_feature_arr = line_ohe.fit_transform(
                              f_tdata[['Airline']]).toarray()
line_feature_labels = list(line_le.classes_ )

line_features = pd.DataFrame(line_feature_arr,
                            columns=line_feature_labels)
f_tdata= pd.concat([f_tdata,line_features],axis=1)
src_le = LabelEncoder()
src_labels = src_le.fit_transform(f_tdata['Source'])
src_ohe=OneHotEncoder()

src_feature_arr = src_ohe.fit_transform(
                              f_tdata[['Source']]).toarray()
src_feature_labels=list(src_le.classes_)
src_features = pd.DataFrame(src_feature_arr,
                            columns=src_feature_labels)
f_tdata= pd.concat([f_tdata,src_features],axis=1)
dest_le = LabelEncoder()
dest_labels = dest_le.fit_transform(f_tdata['Destination'])
dest_ohe=OneHotEncoder()
dest_feature_arr = dest_ohe.fit_transform(
                              f_tdata[['Destination']]).toarray()
dest_feature_labels = list(dest_le.classes_+" Destination")
dest_features = pd.DataFrame(dest_feature_arr,
                            columns=dest_feature_labels)
f_tdata= pd.concat([f_tdata,dest_features],axis=1)
stops_le = LabelEncoder()
stops_labels = stops_le.fit_transform(f_tdata['Total_Stops'])
stops_ohe=OneHotEncoder()
stops_feature_arr = stops_ohe.fit_transform(
                              f_tdata[['Total_Stops']]).toarray()
stops_feature_labels = list(stops_le.classes_)
stops_features = pd.DataFrame(stops_feature_arr,
                            columns=stops_feature_labels)
f_tdata= pd.concat([f_tdata,stops_features],axis=1)

add_le = LabelEncoder()
add_labels = add_le.fit_transform(f_tdata['Additional_Info'])
add_ohe=OneHotEncoder()
add_feature_arr = add_ohe.fit_transform(
                              f_tdata[['Additional_Info']]).toarray()
add_feature_labels = list(add_le.classes_)
add_features = pd.DataFrame(add_feature_arr,
                            columns=add_feature_labels)
f_tdata= pd.concat([f_tdata,add_features],axis=1)

/media/adit/5d04464b-934c-455c-af4f-f0e6a43d237d/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [82]:
doj_ohe=OneHotEncoder()
doj_feature_arr = doj_ohe.fit_transform(
                              dft[['Date_of_Journey']]).toarray()
doj_features = pd.DataFrame(doj_feature_arr,
                            columns=doj_feature_labels)
dft= pd.concat([dft,doj_features],axis=1)


line_le = LabelEncoder()
line_labels = line_le.fit_transform(dft['Airline'])
line_ohe=OneHotEncoder()
line_feature_arr = line_ohe.fit_transform(
                              dft[['Airline']]).toarray()
line_feature_labels = list(line_le.classes_ )

line_features = pd.DataFrame(line_feature_arr,
                            columns=line_feature_labels)
dft= pd.concat([dft,line_features],axis=1)

src_ohe=OneHotEncoder()
src_feature_arr = src_ohe.fit_transform(
                              dft[['Source']]).toarray()
src_feature_labels = list(src_le.classes_ + " Source")
src_features = pd.DataFrame(src_feature_arr,
                            columns=src_feature_labels)
dft= pd.concat([dft,src_features],axis=1)

dest_ohe=OneHotEncoder()
dest_feature_arr = dest_ohe.fit_transform(
                              dft[['Destination']]).toarray()
dest_feature_labels = list(dest_le.classes_+" Destination")
dest_features = pd.DataFrame(dest_feature_arr,
                            columns=dest_feature_labels)
dft= pd.concat([dft,dest_features],axis=1)

stops_ohe=OneHotEncoder()
stops_feature_arr = stops_ohe.fit_transform(
                              dft[['Total_Stops']]).toarray()
stops_feature_labels = list(stops_le.classes_)
stops_features = pd.DataFrame(stops_feature_arr,
                            columns=stops_feature_labels)
dft= pd.concat([dft,stops_features],axis=1)
add_le = LabelEncoder()
add_labels = add_le.fit_transform(dft['Additional_Info'])
add_ohe=OneHotEncoder()
add_feature_arr = add_ohe.fit_transform(
                              dft[['Additional_Info']]).toarray()
add_feature_labels = list(add_le.classes_)
add_features = pd.DataFrame(add_feature_arr,
                            columns=add_feature_labels)
dft= pd.concat([dft,add_features],axis=1)

/media/adit/5d04464b-934c-455c-af4f-f0e6a43d237d/conda/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:368: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


## Fixing Columns of the testing dataset.

In [87]:
data.columns.difference(dft.columns)
len(dft_data.columns)

78

In [84]:
columns=data.columns.difference(dft.columns)
for i in columns:
    dft[i]=0

In [98]:
data= f_tdata.drop(['Airline','Total_Stops','Source','Date_of_Journey','Destination','Route','Additional_Info','Duration','Price','Dep_Time','Arrival_Time'],axis=1)
dft_data= dft.drop(['Airline','Total_Stops','Source','Date_of_Journey','Destination','Route','Additional_Info','Duration','Dep_Time','Arrival_Time', 'Banglore', 'Chennai', 'Delhi', 'Kolkata', 'Mumbai'],axis=1)
target=f_tdata['Price']

## Creating a linear model using SGD Regressor and splitting the data set into training and testing set.

In [88]:
clf = linear_model.SGDRegressor(max_iter=1000)
data_train, data_test, target_train, target_test = train_test_split(data,target, test_size = 0.30, random_state = 10)

### Fiting the test data

In [99]:
clf.fit(data, target)

/media/adit/5d04464b-934c-455c-af4f-f0e6a43d237d/conda/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


SGDRegressor(alpha=0.0001, average=False, early_stopping=False, epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.15,
       learning_rate='invscaling', loss='squared_loss', max_iter=1000,
       n_iter=None, n_iter_no_change=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, tol=None, validation_fraction=0.1,
       verbose=0, warm_start=False)

### Calculating R squared score for the model

In [100]:
clf.score(data_test,target_test)

0.747514627260193

## Predicting for the test data and saving it as a xlsx file.

In [101]:
precti=clf.predict(dft_data)

In [102]:
df=pd.DataFrame(precti)
df.to_excel("output.xlsx")